In [ ]:
# This Python 3 environment comes with many helpful analytics libraries installed
# It is defined by the kaggle/python Docker image: https://github.com/kaggle/docker-python
# For example, here's several helpful packages to load

import numpy as np # linear algebra
import pandas as pd # data processing, CSV file I/O (e.g. pd.read_csv)

# Input data files are available in the read-only "../input/" directory
# For example, running this (by clicking run or pressing Shift+Enter) will list all files under the input directory

import os
for dirname, _, filenames in os.walk('/kaggle/input'):
    for filename in filenames:
        print(os.path.join(dirname, filename))

# You can write up to 20GB to the current directory (/kaggle/working/) that gets preserved as output when you create a version using "Save & Run All" 
# You can also write temporary files to /kaggle/temp/, but they won't be saved outside of the current session

In [34]:
!pip install pandas
!pip install numpy
import pandas as pd
import numpy as np
from tensorflow.keras.preprocessing.image import ImageDataGenerator
from tensorflow.keras.applications.mobilenet_v2 import MobileNetV2
from tensorflow.keras.layers import GlobalAveragePooling2D, Dense
from tensorflow.keras.models import Model
from sklearn.preprocessing import LabelEncoder
from tensorflow.keras.utils import to_categorical

# Set the path to the dataset files
dataset_path = '/kaggle/input/fashion-product-images-dataset/fashion-dataset'

# Load the style.csv file
df = pd.read_csv(dataset_path + '/styles.csv', usecols=['id', 'baseColour', 'articleType', 'season', 'gender'], on_bad_lines='skip')

# Filter out rows with missing values
df = df.dropna()

# Convert 'id' column to string data type and modify to include the image file extension
df['id'] = df['id'].astype(str) + '.jpg'

# Encode categorical labels
label_encoder = LabelEncoder()
df['baseColour'] = label_encoder.fit_transform(df['baseColour'])
df['articleType'] = label_encoder.fit_transform(df['articleType'])
df['season'] = label_encoder.fit_transform(df['season'])
df['gender'] = label_encoder.fit_transform(df['gender'])

# Split the dataset into training and testing sets
train_ratio = 0.8
train_size = int(len(df) * train_ratio)
train_df = df[:train_size]
test_df = df[train_size:]

# Preprocess the image data
image_size = (224, 224)  # Adjust the size as needed

# Define an image data generator
datagen = ImageDataGenerator(rescale=1./255)

# Prepare the training data
train_generator = datagen.flow_from_dataframe(
    dataframe=train_df,
    directory=dataset_path + '/images',
    x_col='id',
    y_col=['baseColour', 'articleType', 'season', 'gender'],
    target_size=image_size,
    batch_size=32,
    class_mode='raw')

# Prepare the testing data
test_generator = datagen.flow_from_dataframe(
    dataframe=test_df,
    directory=dataset_path + '/images',
    x_col='id',
    y_col=['baseColour', 'articleType', 'season', 'gender'],
    target_size=image_size,
    batch_size=32,
    class_mode='raw')






Found 35525 validated image filenames.


/opt/conda/lib/python3.10/site-packages/keras/preprocessing/image.py:1137: UserWarning: Found 3 invalid image filename(s) in x_col="id". These filename(s) will be ignored.
  warnings.warn(


Found 8880 validated image filenames.


/opt/conda/lib/python3.10/site-packages/keras/preprocessing/image.py:1137: UserWarning: Found 2 invalid image filename(s) in x_col="id". These filename(s) will be ignored.
  warnings.warn(


In [35]:
for column in ['baseColour', 'articleType', 'season', 'gender']:
    unique_values = train_df[column].unique()
    num_unique_values = len(unique_values)
    num_classes = len(train_df[column].unique())
    print(f"{column}: Number of unique values: {num_unique_values}, Number of classes: {num_classes}")


baseColour: Number of unique values: 46, Number of classes: 46
articleType: Number of unique values: 140, Number of classes: 140
season: Number of unique values: 4, Number of classes: 4
gender: Number of unique values: 5, Number of classes: 5


In [36]:
# Calculate the number of classes for each label
num_classes = [train_df[column].nunique() for column in ['baseColour', 'articleType', 'season', 'gender']]



# Print the unique values and number of classes for each label
for i, column in enumerate(['baseColour', 'articleType', 'season', 'gender']):
    unique_values = train_df[column].unique()
    print(f"{column}: Unique values: {unique_values}")
    print(f"{column}: Number of classes: {num_classes[i]}")




baseColour: Unique values: [25  2 37  1 13 12 32 44  0  4  3 42  8 31 27 19 33 15 29  7 45  6 11 39
 40 22 18 16 36  9 30 28 38  5 14 35 34 17 20 43 21 24 41 26 23 10]
baseColour: Number of classes: 46
articleType: Unique values: [104  56 140 128 134 110  19   7  39  48 127  13 100 105 120  27  41  14
  72  38  63 138 112 107  15 101  91  51 116  53  90  84  65 102  92  28
  87 109 139   8  94  62  22 108   2  18 132  31  16  12  58  30  17  69
   5  82 135  55 133  73  37  85  29 111  42 119  44 131 129 121 106  43
   3  88 123   1  66  52 130  64  81  80  68  34  23  32   0   6  59  60
 141 118  70  95  99  89 113  33  50 125  83  20  75  35  74  45  78 142
 122 124  40 114 103  86  98  25  57  47  24  96   9 117  11 137  46  97
   4  67  21  77 126  26  61  76  71 136  36  49  79  93]
articleType: Number of classes: 140
season: Unique values: [0 2 3 1]
season: Number of classes: 4
gender: Unique values: [2 4 0 1 3]
gender: Number of classes: 5


In [38]:
# # Convert labels to categorical format

train_labels = [to_categorical(train_generator.labels[:, i]) for i in range(4)]
test_labels = [to_categorical(test_generator.labels[:, i]) for i in range(4)]


# # Convert labels to categorical format
# num_classes = [len(train_df[column].unique()) for column in ['baseColour', 'articleType', 'season', 'gender']]
# num_classes[1] = 50  # Update the number of classes for "articleType" column

# train_labels = [to_categorical(train_generator.labels[:, i], num_classes[i]) for i in range(4)]
# test_labels = [to_categorical(test_generator.labels[:, i], num_classes[i]) for i in range(4)]

# You can use weights='Imagenet' or manually download mobilenet v2 h5 file and load it 

In [48]:
# Build the deep learning model
base_model = MobileNetV2(weights='/kaggle/input/mobilenetv2/mobilenet_v2_weights_tf_dim_ordering_tf_kernels_1.0_224_no_top.h5', include_top=False, input_shape=(224, 224, 3))
x = base_model.output
x = GlobalAveragePooling2D()(x)


# Define output layers for each label
output_baseColour = Dense(num_classes[0], activation='softmax')(x)
output_articleType = Dense(num_classes[1], activation='softmax')(x)
output_season = Dense(num_classes[2], activation='softmax')(x)
output_gender = Dense(num_classes[3], activation='softmax')(x)
print("Color shape: ", output_baseColour.shape)
print("Article type shape: ", output_articleType.shape)
print("Season shape: ", output_season.shape)
print("Gender shape: ", output_gender.shape)
# Create the model
model = Model(inputs=base_model.input, outputs=[output_baseColour, output_articleType, output_season, output_gender])

# Compile the model with appropriate loss functions
losses = ['categorical_crossentropy'] * 4  # Use categorical cross-entropy for each output
model.compile(optimizer='adam', loss=losses, metrics=['accuracy'])


Color shape:  (None, 46)
Article type shape:  (None, 140)
Season shape:  (None, 4)
Gender shape:  (None, 5)


In [ ]:
# Train the model
model.fit(train_generator, validation_data=test_generator,epochs=10)

In [ ]:
from tensorflow.keras.models import load_model
from tensorflow.keras.preprocessing import image

# Path to the saved model file
model_path = "path_to_saved_model.h5"  # Replace with the actual model path

# Load the model
model = load_model(model_path)

# Path to the fashion product image you want to predict
image_path = "path_to_image.jpg"  # Replace with the actual image path

# Load the image and preprocess it
img = image.load_img(image_path, target_size=image_size)
img_array = image.img_to_array(img)
img_array = np.expand_dims(img_array, axis=0)
img_array /= 255.0  # Normalize the image

# Make predictions
predictions = model.predict(img_array)

# Decode the predictions
decoded_predictions = []
for i in range(len(predictions)):
    decoded_predictions.append(label_encoder[i].inverse_transform(np.argmax(predictions[i])))

# Print the predicted labels
print("Predicted base colour:", decoded_predictions[0])
print("Predicted article type:", decoded_predictions[1])
print("Predicted season:", decoded_predictions[2])
print("Predicted gender:", decoded_predictions[3])
